In [32]:
# The necessities
import pandas as pd
from requests import get
import lxml.html as lh

In [33]:
# Headers for our HTTP Requests
headers = {	'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'accept-encoding': 'gzip, deflate, sdch, br',
            'accept-language': 'en-GB,en-US;q=0.8,en;q=0.6',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) Chrome/51.0.2704.79 Safari/537.36',
            'Cache-Control': 'no-cache',
            'Connection': 'keep-alive' }

In [34]:
def parse_and_gather():
    '''This function makes the GET request & collects the data'''
    
    #HTTP GET Call Setup
    url = 'https://www.wilottery.com/lottogames/powerballhistoryOD.aspx'
    response = get(url,headers=headers)
    document = lh.fromstring(response.content)
    table_elements = document.xpath('//tr')
    
    #Define the Column Headers for the 5 Empty Column Headers
    num_list = ['Num_1', 'Num_2', 'Num_3', 'Num_4', 'Num_5']
    
    #Fill in the Column Headers
    col_header = [i.text_content() for i in table_elements[0]]
    col = [[i,[]] for i in col_header]
    
    #Fill in the middle 5 empty column headers with num_list
    for i in range(0,len(num_list)):
        j = i+1
        col[j][0] = num_list[i]

    #Beacause the first row is the headers, the data is stored on the second row onwards
    for j in range(1,len(table_elements)):
        T=table_elements[j] 
        
        # If the table has more than the 8 Columns on the WI Lottery Site
        if len(T)!=8:
            break
            
        #Convert the elements into integers and save them to col lists.     
        i=0
        for t in T.iterchildren():
            data=t.text_content() 
            if i>0:
                try:
                    data=int(data)
                except:
                    pass
            col[i][1].append(data)
            i+=1
    return col

In [35]:
def build_dataframe():
    ''' This function creates a Pandas DataFrame from the data'''
    
    #Call the Gather Data Function
    col = parse_and_gather()
    
    #Convert to a Dictionary
    Dict={header:column for (header,column) in col}

    #Create a DataFrame
    df=pd.DataFrame.from_dict(Dict)

    #Set the Date Column as the index
    df.set_index('Draw Date', inplace=True)

    #Convert the Index to a Pandas DatetimeIndex
    df.index = pd.to_datetime(df.index)
    
    #Selects all Non-Digits
    digits = r"\D+"
    #Selects all spaces
    spaces = r"\s+"
    
    #Splits the powerplay column into just the integer values
    df['Power Play_Clean'] = df['Power Play'].astype(str).str.split(spaces).str.get(1).str.split(digits).str.get(0)

    #Drop the old Powerplay column
    df.drop('Power Play', axis=1, inplace=True)

    #Add a new Powerplay column that contains the cleaned information
    df['Power Play'] = pd.to_numeric(df['Power Play_Clean'], errors='coerce', downcast='integer')

    #Drop the Cleaning Column
    df.drop('Power Play_Clean', axis=1, inplace=True)
    
    return df

In [38]:
# Call the Build DataFrame Function
df = build_dataframe()

#Lets see what we have
df.head()

1554